In [8]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import time

# --- 1. Define Paths ---
base_path = "/kaggle/input/animal-ds/"
train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")
image_size = (64, 64)

# --- 2. Helper function to load data (now handles different cat folder names) ---
def load_images_from_folder(folder_path, cat_folder_name="cats"):
    data = []
    labels = []
    # Load Cats (label 0)
    cat_folder = os.path.join(folder_path, cat_folder_name) # Use the provided name
    for img_name in os.listdir(cat_folder):
        try:
            img_path = os.path.join(cat_folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_resized = cv2.resize(img, image_size)
            data.append(img_resized)
            labels.append(0) # 0 for Cat
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

    # Load Dogs (label 1) - assuming 'dogs' is the same name in both
    dog_folder = os.path.join(folder_path, "dogs")
    for img_name in os.listdir(dog_folder):
        try:
            img_path = os.path.join(dog_folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_resized = cv2.resize(img, image_size)
            data.append(img_resized)
            labels.append(1) # 1 for Dog
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            
    return np.array(data), np.array(labels)

# --- 3. Load Training and Test Data ---
print("Loading TRAINING images (using 'cats')...")
# Use "cats" for the train path
X_train_img, y_train = load_images_from_folder(train_path, cat_folder_name="cats") 
print(f"Training data shape: {X_train_img.shape}")

print("Loading TEST images (using 'cat_s')...")
# Use "cat_s" for the test path
X_test_img, y_test = load_images_from_folder(test_path, cat_folder_name="cat_s") 
print(f"Test data shape: {X_test_img.shape}")

# --- 4. Manual Feature Extraction (HOG) ---
print("Extracting HOG features for TRAINING data...")
hog_features_train = []
for image in X_train_img:
    features = hog(image, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), visualize=False)
    hog_features_train.append(features)
hog_features_train = np.array(hog_features_train)

print("Extracting HOG features for TEST data...")
hog_features_test = []
for image in X_test_img:
    features = hog(image, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), visualize=False)
    hog_features_test.append(features)
hog_features_test = np.array(hog_features_test)

# --- 5. Train Traditional ML Model (SVM) ---
print(f"Training SVM on {len(hog_features_train)} samples...")
start_time = time.time()
model_svm = SVC(kernel='linear', C=1.0)
model_svm.fit(hog_features_train, y_train)
end_time = time.time()
print(f"SVM training finished in {end_time - start_time:.2f} seconds.")

# --- 6. Evaluate the Traditional ML Model ---
print("\n--- Traditional ML (SVM + HOG) Results ---")
predictions = model_svm.predict(hog_features_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(classification_report(y_test, predictions, target_names=['Cat', 'Dog']))

Loading TRAINING images (using 'cats')...
Training data shape: (555, 64, 64)
Loading TEST images (using 'cat_s')...
Test data shape: (138, 64, 64)
Extracting HOG features for TRAINING data...
Extracting HOG features for TEST data...
Training SVM on 555 samples...
SVM training finished in 0.21 seconds.

--- Traditional ML (SVM + HOG) Results ---
Accuracy: 65.94%
              precision    recall  f1-score   support

         Cat       0.65      0.70      0.67        69
         Dog       0.67      0.62      0.65        69

    accuracy                           0.66       138
   macro avg       0.66      0.66      0.66       138
weighted avg       0.66      0.66      0.66       138



In [10]:
pip install protobuf==3.20.3

ERROR: Could not find a version that satisfies the requirement protobuf==3.20.3 (from versions: none)
ERROR: No matching distribution found for protobuf==3.20.3
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import cv2
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import time
import tensorflow as tf
from tensorflow.keras import layers, models

# --- 1. Define Paths ---
base_path = "/kaggle/input/animal-ds/"
train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")
image_size = (64, 64)

# --- 2. Helper function to load data (now handles different cat folder names) ---
def load_images_for_cnn(folder_path, cat_folder_name="cats"):
    data = []
    labels = []
    # Load Cats (label 0)
    cat_folder = os.path.join(folder_path, cat_folder_name) # Use the provided name
    for img_name in os.listdir(cat_folder):
        try:
            img_path = os.path.join(cat_folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR) 
            img_resized = cv2.resize(img, image_size)
            data.append(img_resized)
            labels.append(0) # 0 for Cat
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

    # Load Dogs (label 1)
    dog_folder = os.path.join(folder_path, "dogs")
    for img_name in os.listdir(dog_folder):
        try:
            img_path = os.path.join(dog_folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            img_resized = cv2.resize(img, image_size)
            data.append(img_resized)
            labels.append(1) # 1 for Dog
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            
    data_np = np.array(data).astype('float32') / 255.0
    labels_np = np.array(labels)
    return data_np, labels_np

# --- 3. Load Training and Test Data ---
print("Loading TRAINING images for CNN (using 'cats')...")
# Use "cats" for the train path
X_train_cnn, y_train_cnn = load_images_for_cnn(train_path, cat_folder_name="cats")
print(f"Training data shape: {X_train_cnn.shape}")

print("Loading TEST images for CNN (using 'cat_s')...")
# Use "cat_s" for the test path
X_test_cnn, y_test_cnn = load_images_for_cnn(test_path, cat_folder_name="cat_s")
print(f"Test data shape: {X_test_cnn.shape}")

# --- 4. Build the Deep Learning Model (CNN) ---
print("Building CNN model...")
model_cnn = models.Sequential()
model_cnn.add(layers.Input(shape=(64, 64, 3)))
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='relu'))
model_cnn.add(layers.Dense(1, activation='sigmoid')) 

model_cnn.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
model_cnn.summary()

# --- 5. Train the Deep Learning Model ---
print("Training CNN model...")
start_time = time.time()
model_cnn.fit(X_train_cnn, y_train_cnn, epochs=10, 
              validation_data=(X_test_cnn, y_test_cnn))
end_time = time.time()
print(f"CNN training finished in {end_time - start_time:.2f} seconds.")

# --- 6. Evaluate the Deep Learning Model ---
print("\n--- Deep Learning (CNN) Results ---")
test_loss, test_acc = model_cnn.evaluate(X_test_cnn, y_test_cnn)
print(f"Accuracy: {test_acc * 100:.2f}%")

predictions_prob = model_cnn.predict(X_test_cnn)
predictions_cnn = (predictions_prob > 0.5).astype(int)
print(classification_report(y_test_cnn, predictions_cnn, target_names=['Cat', 'Dog']))

Loading TRAINING images for CNN (using 'cats')...
Training data shape: (555, 64, 64, 3)
Loading TEST images for CNN (using 'cat_s')...
Test data shape: (138, 64, 64, 3)
Building CNN model...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 822,337 (3.14 MB)

 Trainable params: 822,337 (3.14 MB)

 Non-trainable params: 0 (0.00 B)

Training CNN model...
Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 100ms/step - accuracy: 0.4857 - loss: 0.8432 - val_accuracy: 0.4928 - val_loss: 0.6922
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5006 - loss: 0.6939 - val_accuracy: 0.5000 - val_loss: 0.6899
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5966 - loss: 0.6776 - val_accuracy: 0.5072 - val_loss: 0.6892
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5653 - loss: 0.6863 - val_accuracy: 0.5797 - val_loss: 0.6818
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7344 - loss: 0.6186 - val_accuracy: 0.6304 - val_loss: 0.6610
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7575 - loss: 0.5379 - val_accuracy: 0.6014 - val_loss: 0.6917
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7937 - loss: 0.4617 - val_accuracy: 0.6159 - val_loss: 0.7189
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8441 - loss: 0.3933 - v

In [3]:
!ls /kaggle/input/animal-ds/train/

cats  dogs


In [19]:

base_path = "/kaggle/input/animal-ds/"